# Chapter 19: Time Series Forecasting

Notebook ini merupakan hasil reproduksi dan penjelasan teori dari **Bab 19 - Time Series Forecasting** dari buku *Hands-On Machine Learning with Scikit-Learn, Keras, and TensorFlow (2nd Edition)* oleh Aurélien Géron.

📌 Bab ini membahas pendekatan machine learning dan deep learning untuk memprediksi data deret waktu, seperti suhu, harga saham, dan sinyal.

---


## Ringkasan Teori Bab 19: Time Series Forecasting

### 1. Apa itu Time Series?

- Urutan data yang diindeks menurut waktu
- Contoh: suhu harian, penjualan bulanan, nilai saham

---

### 2. Tujuan Forecasting

- Memprediksi nilai masa depan berdasarkan data masa lalu
- Aplikasi: supply chain, energi, keuangan, cuaca

---

### 3. Tantangan Deret Waktu

- Data sering memiliki:
  - **Trend**: pola jangka panjang naik/turun
  - **Seasonality**: pola berulang periodik
  - **Noise**: fluktuasi acak
- Forecasting harus mempertimbangkan semua komponen ini

---

### 4. Teknik Forecasting Sederhana

- **Naive Forecast**: nilai berikutnya = nilai sekarang
- **Moving Average**: rata-rata nilai sebelumnya
- **Differencing**: untuk menghilangkan trend/seasonality

---

### 5. Evaluasi Performa

- MSE, MAE
- Evaluasi dilakukan di data validasi (forecasting horizon)

---

### 6. Persiapan Dataset

- Buat window sliding: input = data historis, target = nilai selanjutnya
- Gunakan batching dan prefetch untuk efisiensi

```python
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    ...
```

---

### 7. Model Neural Network

- Gunakan model:
  - **Dense** (feedforward)
  - **RNN/LSTM** untuk temporal pattern
  - **1D CNN** untuk pola lokal
- Aktivasi dan loss:
  - `relu`, `mse` atau `mae`

---

### 8. Strategi Forecasting

- **One-step ahead**: prediksi 1 langkah ke depan
- **Recursive**: hasil prediksi dipakai sebagai input berikutnya
- **Sequence-to-Sequence**: prediksi beberapa langkah sekaligus

---

📌 Ringkasan:
Forecasting deret waktu adalah masalah klasik yang bisa diselesaikan dengan teknik statistik atau neural network. Pemrosesan window dan validasi berperan penting dalam akurasi prediksi.

---


## Implementasi Model LSTM untuk Time Series Forecasting

Kita akan menggunakan data suhu buatan (sinusoidal) untuk:
- Membuat dataset time series
- Membentuk window (sequence) untuk input
- Melatih model LSTM sederhana untuk memprediksi suhu berikutnya

---


In [ ]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt

# Generate dummy time series (suhu sinusoidal)
time = np.arange(365 * 2)
series = 10 + np.sin(time * 2 * np.pi / 365) + np.random.normal(0, 0.3, size=len(time))

plt.plot(time, series)
plt.title("Suhu Tahunan (Sinusoidal)")
plt.xlabel("Hari")
plt.ylabel("Suhu")
plt.grid(True)
plt.show()

In [ ]:
# Fungsi windowing dataset
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True)
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1))
    dataset = dataset.shuffle(shuffle_buffer)
    dataset = dataset.map(lambda window: (window[:-1], window[-1]))
    return dataset.batch(batch_size).prefetch(1)

# Siapkan dataset
window_size = 30
batch_size = 32
shuffle_buffer = 1000

split_time = 500
train_series = series[:split_time]
val_series = series[split_time:]

train_set = windowed_dataset(train_series, window_size, batch_size, shuffle_buffer)
val_set = windowed_dataset(val_series, window_size, batch_size, shuffle_buffer)


In [ ]:
# Buat dan latih model LSTM
model = tf.keras.models.Sequential([
    tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1), input_shape=[window_size]),
    tf.keras.layers.LSTM(32, return_sequences=False),
    tf.keras.layers.Dense(1)
])

model.compile(loss="mse", optimizer=tf.keras.optimizers.Adam(learning_rate=0.001))
history = model.fit(train_set, epochs=20, validation_data=val_set)


In [ ]:
# Visualisasi loss
plt.plot(history.history["loss"], label="Train Loss")
plt.plot(history.history["val_loss"], label="Val Loss")
plt.title("Training vs Validation Loss")
plt.xlabel("Epoch")
plt.ylabel("Loss")
plt.legend()
plt.grid(True)
plt.show()

### Penjelasan:
- Dataset sinusoidal merepresentasikan pola musiman suhu
- `windowed_dataset()` membentuk input `X` dari 30 hari sebelumnya, target `y` = suhu hari ke-31
- Model LSTM mempelajari pola temporal
- Visualisasi menunjukkan performa model pada data validasi

📌 Ini adalah dasar dari banyak aplikasi forecasting seperti harga saham, energi, atau cuaca.

---
